# Packages 

In [3]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import scipy
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *


/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [4]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [5]:
debug = False

eval_cg = False

reprocess_train_data = False
train_parts = 20

load_train_eval_data = False
if debug:
    SAMPLE_NUM = 10000
else:
    SAMPLE_NUM = None


candidate_path = '../data/candidates/'
model_dir = '../model_training'
ranker_train_data_dir = '../data/rank_train_data_v2'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model_version = 'w2v_v3'
nic_model_version = 'nic'
nfi_model_version = 'next_few_items_v1'
covisit_model_version = 'co_visit'

# rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
nfi_topn=100
covisit_topn = 300
# PREDS_PER_SESSION = 100

# num_tree = 100
# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")



train_dir = os.path.join(ranker_train_data_dir, 'train')

In [6]:
! mkdir {ranker_train_data_dir}

mkdir: cannot create directory ‘../data/rank_train_data_v2’: File exists


In [7]:
! mkdir {train_dir}

mkdir: cannot create directory ‘../data/rank_train_data_v2/train’: File exists


In [8]:
train_dir

'../data/rank_train_data_v2/train'

# Analysis

In [11]:
eval_cg_pl = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))


In [18]:
assert eval_cg_pl.select(pl.col('session_id')).unique().collect().shape[0] == eval_pl.select(pl.col('session_id')).unique().collect().shape[0]

In [19]:
eval_cg_pl.head().collect()

session_id,next_item_prediction,w2v_weight,next_item,locale,prev_length,target,next_item_weight,next_item_weight_nfi,next_item_weight_co_visit,last_item_similarity
i64,str,f64,str,i32,u32,i32,f64,f32,f32,f32
3,"""B09P1MBQY4""",0.525773,"""B0B4R9NN4B""",1,7,0,0.0,0.1,2.0,0.987218
3,"""B0B2D37NXD""",0.0,"""B0B4R9NN4B""",1,7,0,0.0,0.0,1.0,0.994865
3,"""B0B4D56ZMW""",0.0,"""B0B4R9NN4B""",1,7,0,0.0,0.33,1.0,0.994023
3,"""B0B833FRKK""",0.597938,"""B0B4R9NN4B""",1,7,0,0.0,0.0,0.0,0.990813
3,"""B0B3965NRQ""",0.0,"""B0B4R9NN4B""",1,7,0,0.0,0.07,1.0,0.978123


## recall=1 or 0 

In [22]:
combined_pl = (
    eval_cg_pl.groupby('session_id')
        .agg(pl.col('next_item_prediction'))
        .join(
            eval_pl.select(['session_id', 'next_item'])
            , how='left'
            , on='session_id'
             )
)

combined_1 = combined_pl.filter(pl.col('next_item_prediction').arr.contains(pl.col('next_item')))
combined_0 = combined_pl.filter(pl.col('next_item_prediction').arr.contains(pl.col('next_item')) == False)

In [31]:
(
        combined_1.select(
                pl.col('next_item_prediction').arr.lengths().alias('rec_num')
            )
            .collect()
            .describe(
                # percentiles=[0.1*i for i in range(10)]
            )
    )

describe,rec_num
str,f64
"""count""",199876.0
"""null_count""",0.0
"""mean""",206.523955
"""std""",97.175547
"""min""",72.0
"""max""",478.0
"""median""",170.0


In [32]:
(
        combined_0.select(
                pl.col('next_item_prediction').arr.lengths().alias('rec_num')
            )
            .collect()
            .describe(
                # percentiles=[0.1*i for i in range(10)]
            )
    )

describe,rec_num
str,f64
"""count""",126567.0
"""null_count""",0.0
"""mean""",180.73083
"""std""",94.73839
"""min""",66.0
"""max""",478.0
"""median""",137.0


## case study 

In [33]:
combined_1.head().collect()

session_id,next_item_prediction,next_item
i64,list[str],str
413344,"[""B0BGPRRJDP"", ""B08FRN4N1M"", … ""B0BGPPZG2R""]","""B0BHX55BHS"""
2483552,"[""B09WF4MM8F"", ""B084ZPL21L"", … ""B07L3LJHVC""]","""B09G2TK6LS"""
1563936,"[""B09139Z9L8"", ""B09YVN886F"", … ""B07Y9X7VBX""]","""B07GFF82DP"""
2851552,"[""B0B8RJK4R8"", ""B07KBFZ58N"", … ""B07TS6QRTF""]","""B08NTBFF1G"""
165280,"[""B0B11GLKQJ"", ""B0BG5TS7C2"", … ""B07LFRP5SS""]","""B0B8Z9N114"""


In [39]:
session_id = 165280

In [40]:
eval_cg_pl.filter(pl.col('session_id')==session_id).sort('target', descending=True).collect()

session_id,next_item_prediction,w2v_weight,next_item,locale,prev_length,target,next_item_weight,next_item_weight_nfi,next_item_weight_co_visit,last_item_similarity
i64,str,f64,str,i32,u32,i32,f64,f32,f32,f32
165280,"""B0B8Z9N114""",0.989691,"""B0B8Z9N114""",1,3,1,0.097591,16.919998,30.0,0.989086
165280,"""B0B11GLKQJ""",0.030928,"""B0B8Z9N114""",1,3,0,0.0,0.7,4.0,0.93842
165280,"""B0BG5TS7C2""",0.0,"""B0B8Z9N114""",1,3,0,0.0,0.25,3.0,0.858037
165280,"""B0B9ZZ1RXJ""",0.0,"""B0B8Z9N114""",1,3,0,0.01,1.22,5.0,0.933631
165280,"""B0B8ZCQ38T""",0.917526,"""B0B8Z9N114""",1,3,0,0.031898,6.25,14.0,0.983005
165280,"""B01D4MAH08""",0.0,"""B0B8Z9N114""",1,3,0,0.01,1.0,2.0,0.912953
165280,"""B09LYWG5XR""",0.0,"""B0B8Z9N114""",1,3,0,0.0,0.25,2.0,0.924414
165280,"""B0BGSRWVL3""",0.0,"""B0B8Z9N114""",1,3,0,0.0,0.58,6.0,0.965941
165280,"""B0B65JNJQX""",0.0,"""B0B8Z9N114""",1,3,0,0.0,0.28,2.0,0.821898


# Function 

In [6]:
def candidates_validation(target_pl):
    # rec_summary  = (target_pl
    #         .lazy()
    #         .select(
    #             pl.col('next_item_prediction').arr.lengths().alias('rec_num')
    #         ).collect().describe())
    # print(rec_summary)
    max_sample_per_NIP = (
        target_pl.groupby(['session_id', 'next_item_prediction'])
            .agg(
                pl.count().alias('unique_count')
                # , pl.col('w2v_weight').max().alias('max_w2v_weight')
                # , pl.col('w2v_weight').min().alias('min_w2v_weight')
            )
            .select(
                pl.col('unique_count').max().alias('max_unique_cnt')
                # , pl.col('max_w2v_weight').min().alias('min_max_w2v')
                # , pl.col('min_w2v_weight').max().alias('max_min_w2v')
            )
    ).to_pandas()['max_unique_cnt'].values[0]
    # print(stat_info)
    # print(max_sample_per_NIP)
    assert max_sample_per_NIP == 1, f"there should be unique prediction for every session_id"

# Original data 

In [7]:


# # df_sess.head(3).collect()
# test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))


# Re-process data for memmory 

In [8]:
def re_process_data(task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                    , train_parts=train_parts
                      ):
    parts = train_parts

    data_type = 'train'
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    nfi_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nfi_model_version
                , model_for_eval=model_for_eval
                , topn=nfi_topn
                       ))
    covisit_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=covisit_model_version
                , model_for_eval=model_for_eval
                , topn=covisit_topn
                       ))
    # w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    dir_lst = []
    pl_lst = []
    for file_name in [w2v_file, nic_file, nfi_file, covisit_file]:
        data_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM).collect().sort('session_id')
        dir_name = file_name.replace('.parquet', '')
        print(dir_name)
        dir_lst.append(dir_name)
        pl_lst.append(data_pl)
        if not os.path.isdir(dir_name):
            os.mkdir(dir_name)
    assert pl_lst[0].shape[0] == pl_lst[1].shape[0] == pl_lst[2].shape[0] == pl_lst[3].shape[0]
    total_len = pl_lst[0].shape[0]

    rows_per_part = int(total_len/parts)

    for dir_path, data_pl in zip(dir_lst, pl_lst):
        print(dir_path)
        for idx in tqdm(range(parts)):
            begin = idx*rows_per_part
            if idx != parts-1:
                end = (idx+1)*rows_per_part
            else:
                end = total_len
            part_pl = data_pl[begin:end :]
            file_name = os.path.join(dir_path, f"part_{idx+1}.parquet")
            print(f'{begin} - {end} -> {file_name}')
            part_pl.write_parquet(file_name)

In [9]:
if reprocess_train_data:
    re_process_data()

In [10]:
# ! ls ../data/candidates/task1_train_w2v_v3_True_top100

In [11]:
# df = pl.scan_parquet(os.path.join('../data/candidates/task1_train_co_visit_True_top300/', 'part_20.parquet'))

In [12]:
# df.head().collect()

# Get candiadtes

In [13]:
# data_type = 'eval'
# w2v_file = os.path.join(candidate_path, 
#                            candidate_file_name.format(
#                     task=task
#                 , data_type=data_type
#                 , model_version=w2v_model_version
#                 , model_for_eval=model_for_eval
#                 , topn=w2v_topn
#                            ))
# w2v_file
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)

In [14]:
    # w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


In [15]:
# def similarity_by_vec(x, col='last_prev_item'):
#     try:
#         next_item_prediction = 
#         prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
#         next_item_vec = w2vec.wv[next_item_prediction]
#         1 - scipy.spatial.distance.cosine(prev_items_vec, next_item_vec)
#         simi = w2vec.wv.n_similarity([x['next_item_prediction']],
#                                                   target
#                                                  )
#     except Exception as e:
#         print(e)
#         # print(x[col])
#         simi = 0
#     return simi


def get_all_candidates(data_type, task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                       , part=None
                      ):
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    nfi_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nfi_model_version
                , model_for_eval=model_for_eval
                , topn=nfi_topn
                       ))
    covisit_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=covisit_model_version
                , model_for_eval=model_for_eval
                , topn=covisit_topn
                       ))
    if data_type == 'train':
        assert part is not None
        w2v_file = os.path.join(w2v_file.replace('.parquet', ''), f"part_{part}.parquet")
        nic_file = os.path.join(nic_file.replace('.parquet', ''), f"part_{part}.parquet")
        nfi_file = os.path.join(nfi_file.replace('.parquet', ''), f"part_{part}.parquet")
        covisit_file = os.path.join(covisit_file.replace('.parquet', ''), f"part_{part}.parquet")

    if data_type == 'test':
        original_file_name = f"sessions_{data_type}_{task}.parquet"
    else:
        original_file_name = f"sessions_{data_type}.parquet"
    original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


#     def get_vector(prev_items):
#         prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
#         return prev_items_vec
    
#     original_pl = (
#             original_pl.with_columns(
#                 pl.col('prev_items').apply(lambda x: get_vector(x)).alias('all_previous_items_vec')
#             )
#         )
    
    # get w2v weight
    w2v_pl = w2v_pl.with_columns(pl.col('next_item_prediction').arr.lengths().alias('rec_num')).with_columns(
        pl.col('rec_num').apply(lambda x: list(range(x, 0, -1))).alias('w2v_weight')
        
    )
    # print(w2v_pl.head().collect())
    get_w2v_weight = pl.element().rank()*0
    nic_pl = nic_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                # parallel=True
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    nfi_pl = nfi_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    covisit_pl = covisit_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    cols = ['session_id', 'next_item_prediction', 
            'w2v_weight'
           ]
    # combined_pl = (
    #     w2v_pl.select(cols)
    #         .join(nic_pl.select(cols), how='left', on='session_id', suffix='_nic')
    #         .join(nfi_pl.select(cols), how='left', on='session_id', suffix='_nfi')
    # )

    # combined_pl = combined_pl.with_columns(
    #     pl.concat_list([pl.col('next_item_prediction'), pl.col('next_item_prediction_nic')]).alias('next_item_prediction'), 
    #     pl.concat_list([pl.col('w2v_weight'), pl.col('w2v_weight_nic')]).alias('w2v_weight'), 
    # ).select(
    #     pl.all().exclude(['next_item_prediction_nic', 'w2v_weight_nic'])
    # )
    explode_cols = ['next_item_prediction', 
                    'w2v_weight'
                   ]
    combined_pl = (
            pl.concat([w2v_pl.select(cols).explode(explode_cols)
                       , nic_pl.select(cols).explode(explode_cols)
                       , nfi_pl.select(cols).explode(explode_cols)
                       , covisit_pl.select(cols).explode(explode_cols)
                      ], how='vertical')
                .groupby(['session_id', 'next_item_prediction'])
                .agg(
                    pl.col('w2v_weight').max().alias('w2v_weight')
                )
                .join(original_pl.select(['session_id', 'prev_items']), how='left', on='session_id')
                .filter(pl.col('prev_items').arr.contains(pl.col('next_item_prediction'))==False)
                .with_columns(
                    pl.col('w2v_weight').max().over('session_id').alias('max_count')
                    , pl.col('w2v_weight').min().over('session_id').alias('min_count')
                )
                .with_columns(
                    pl.when(pl.col('max_count')==pl.col('min_count')).then(1).otherwise((pl.col('w2v_weight')-pl.col('min_count'))/(pl.col('max_count')-pl.col('min_count'))).alias('w2v_weight')
                )
                .select(
                        pl.exclude(['prev_items', 'max_count', 'min_count'])

                )
                .groupby('session_id')
                .agg(
                    pl.all()
                )
    )
    combined_pl = (
        combined_pl.join(original_pl, how='left', on='session_id')
            .with_columns(
                pl.col('prev_items').arr.lengths().alias('prev_length')
                , pl.col('prev_items').arr.get(-1).alias('last_prev_item')
            ).select(
                pl.all().exclude('prev_items')
            )
    )
    return combined_pl


In [16]:
eval_cg_pl = get_all_candidates(data_type='eval')


In [17]:
test_cg_pl = get_all_candidates(data_type='test')

In [18]:
# eval_cg_pl.head().collect()

In [19]:
# eval_cg_pl.select(
#     pl.col('next_item_prediction').arr.lengths().alias('len1')
#     , pl.col('w2v_weight').arr.lengths().alias('len2')
# ).collect().describe()

In [20]:
# eval_cg_pl.select(
#     pl.col('next_item_prediction').arr.lengths().alias('len1')
#     , pl.col('w2v_weight').arr.lengths().alias('len2')
# ).collect().describe()

In [21]:
# ).collect().describe()

In [22]:
# eval_cg_pl.head().collect()

In [23]:
# eval_cg_pl.head().collect()

# Eval candidate generation 

In [24]:
if eval_cg:
    eval_topn = 3000

    col = f"recall@{eval_topn}"
    eval_final = (
            eval_cg_pl
            .lazy()
            .with_columns(
                pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
            )
            ).with_columns(
                pl.col('next_item_prediction').arr.head(eval_topn).arr.contains(pl.col('next_item')).mean().alias(col)

            )
    final_res = eval_final.select(
            pl.count().alias('total_sessions')
            , pl.col(col).mean()

        ).collect()
    describe = (eval_cg_pl
            .lazy()
            .select(
                pl.col('next_item_prediction').arr.lengths().alias('rec_num')
            ).collect().describe())
    print(describe)
    print(final_res)
    del final_res

# Feature Process 

## Load NIC, W2V Model 

In [25]:
nic_model = (
    pl.scan_parquet('../model_training/next_item_counter_v2/nic_model.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
        )
            )
print(nic_model.schema)

nfi_model = (
    pl.scan_parquet('../model_training/next_few_items_v1/nic_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
            # , pl.col()
        )
            )
print(nfi_model.schema)



# w2v_model_file = '../model_training/v2/w2v.model'
w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)

{'next_item_prediction': Utf8, 'next_item_weight': Float64, 'last_prev_item': Utf8}
{'next_item_prediction': Utf8, 'next_item_weight': Float32, 'last_prev_item': Utf8}


In [26]:
co_visit_model = (
    pl.scan_parquet(f'../model_training/{covisit_model_version}/{covisit_model_version}_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
            # , pl.col()
        )
            )
print(co_visit_model.schema)

{'next_item_prediction': Utf8, 'next_item_weight': Float32, 'last_prev_item': Utf8}


In [27]:
# co_visit_model.head().collect()

## get features 

In [28]:
# target_df = train_candidates
# data_type = 'train'

def get_w2v_simi(x, col='last_prev_item'):
    try:
        if isinstance(x[col], str):
            target = [x[col]]
        else:
            target = x[col]
        simi = w2vec.wv.n_similarity([x['next_item_prediction']],
                                                  target
                                                 )
    except Exception as e:
        print(e)
        # print(x[col])
        simi = 0
    return simi

def get_feature(target_df, data_type, nic_model=nic_model, nfi_model=nfi_model, 
                co_visit_model=co_visit_model,
                negative_sampling=None, w2v_similarity=False):
    if data_type == 'train':
        target_df = target_df.filter(pl.col('next_item_prediction').arr.contains(pl.col('next_item')))
    target_df = target_df.explode(['next_item_prediction', 'w2v_weight'])

    if data_type != 'test':
        target_df = (
            target_df
                .with_columns(
                    pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
                )
        )

    if negative_sampling is not None:
        cols = ['session_id', 'next_item', 'next_item_prediction', 'target']
        sample_pl = target_df.select(cols)
        pos = sample_pl.filter(pl.col('target')==1)
        neg = (sample_pl
                   .filter(pl.col('target')==0)
              )
        neg_original_len = neg.select('session_id').collect().shape[0]
        print(f'Original negative num: {neg_original_len}')
        neg = (
            neg.with_columns(
                        pl.Series(name='random', values=np.random.uniform(size=neg_original_len))
                   )
                   .filter(pl.col('random')<=negative_sampling)
                   .select(pl.all().exclude('random'))
        )
        # print(neg.shape)
        # print(neg.head().collect())
        sample_pl = pl.concat([pos, neg], how='vertical')
        target_df = sample_pl.join(target_df, how='left', on=cols).collect()
        del sample_pl
    target_df = (
            target_df.lazy().join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
                .join(nfi_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_nfi')
                .join(co_visit_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_co_visit')
                .with_columns(
                    pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
                    , pl.when(pl.col('next_item_weight_nfi').is_null()).then(0).otherwise(pl.col('next_item_weight_nfi')).alias('next_item_weight_nfi')
                    , pl.when(pl.col('next_item_weight_co_visit').is_null()).then(0).otherwise(pl.col('next_item_weight_co_visit')).alias('next_item_weight_co_visit')
                    , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
                        lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
                    # , pl.struct(["next_item_prediction", "prev_items"]).apply(
                    #     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
                    , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                        .then(2)
                        .otherwise(3).alias('locale')
                ).sort('session_id')
                .select(
                    pl.all().exclude('last_prev_item')
                )

    )
    # if w2v_similarity:
    #     target_df = target_df.with_columns(
    #         pl.struct(["next_item_prediction", "prev_items"]).apply(
    #                     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
    #     )
    return target_df
# target_df.head(3).collect()

In [29]:
# %%time 
# sample_pl = get_feature(target_df=train_cg_pl, data_type='train', #negative_sampling=0.1
#                        ).collect()

In [30]:
# sampled_train_candidates.head().collect()

In [31]:
# %%time
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', negative_sampling=0.1)
# debug_pl = sampled_train_candidates.head().collect()

In [32]:
# debug_pl.head()

In [33]:
# %%time
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train',
#                                        negative_sampling=0.1,
#                                       w2v_similarity=True)
# debug_pl = sampled_train_candidates.head().collect()

In [34]:
# debug_pl

In [35]:
# prev_items = ["B09W9FND7K", "B09JSPLN1M"]
# next_item_prediction = "B09M7GY217"	
# w2vec.wv.n_similarity(prev_items, [next_item_prediction])
# w2vec.wv.n_similarity(prev_items, prev_items)
# prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
# next_item_vec = w2vec.wv[next_item_prediction]
# 1 - scipy.spatial.distance.cosine(prev_items_vec, next_item_vec)

# Save Data 

## Save eval & test data 

In [36]:
%load_ext memory_profiler

In [37]:
eval_candidates = get_feature(target_df=eval_cg_pl, data_type='eval')
test_candidates = get_feature(target_df=test_cg_pl, data_type='test')

In [38]:
%%time
%memit
eval_candidates = eval_candidates.collect()

print('Eval data')
print()
print(eval_candidates.shape)
print(f"Memory: {eval_candidates.estimated_size(unit='gb')}")
print(eval_candidates['target'].mean())
print('Validating')
candidates_validation(target_pl=eval_candidates)
print()

peak memory: 1626.96 MiB, increment: 0.10 MiB
Eval data

(64153741, 11)
Memory: 5.033693803474307
0.0031155782481960014
Validating

CPU times: user 56min 36s, sys: 3min 5s, total: 59min 42s
Wall time: 49min 1s


In [ ]:
%%time
%memit

test_candidates = test_candidates.collect()
print('Test data')
print()
print(test_candidates.shape)
print(f"Memory: {test_candidates.estimated_size(unit='gb')}")
print('Validating')
candidates_validation(target_pl=test_candidates)

if not debug:
    eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
    test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))

peak memory: 8605.78 MiB, increment: 0.00 MiB


In [ ]:
# eval_candidates.head()

In [ ]:
# eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
# test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))

## Saving train data 

In [ ]:
# train_cg_pl.with_columns(pl.all().is_null()).sum().collect()

In [ ]:
# train_cg_pl.collect().sample(10)

In [ ]:
if debug:
    train_parts = 1
for part_index in tqdm(range(train_parts)):
    print()
    file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
    print(file_name)
    train_cg_pl = get_all_candidates(data_type='train', part=part_index+1)
    sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
                                           # negative_sampling=0.1
                                          )
    sampled_train_candidates = sampled_train_candidates.collect()
    print('Validating')
    candidates_validation(target_pl=sampled_train_candidates)
    # print('Sampled train data')
    print(sampled_train_candidates.shape)
    print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
    print(sampled_train_candidates['target'].mean())
    
    stat_info = (
            sampled_train_candidates
                .groupby('session_id')
                .agg(
                    pl.count().alias('rec_cnt')
                    , pl.col('target').sum().alias('target_num')
                )
                .describe()
        )
    print(stat_info)
    if not debug:
        sampled_train_candidates.write_parquet(file_name)
    # del sampled_train_candidates

  0%|          | 0/20 [00:00<?, ?it/s]


../data/rank_train_data_v2/train/part_1.parquet
Validating
(4967840, 11)
Memory: 0.38863958418369293
0.010297835679087894
shape: (7, 4)
┌────────────┬──────────────┬───────────┬────────────┐
│ describe   ┆ session_id   ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---          ┆ ---       ┆ ---        │
│ str        ┆ f64          ┆ f64       ┆ f64        │
╞════════════╪══════════════╪═══════════╪════════════╡
│ count      ┆ 51158.0      ┆ 51158.0   ┆ 51158.0    │
│ null_count ┆ 0.0          ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 81732.982916 ┆ 97.107784 ┆ 1.0        │
│ std        ┆ 47181.001648 ┆ 1.794644  ┆ 0.0        │
│ min        ┆ 5.0          ┆ 74.0      ┆ 1.0        │
│ max        ┆ 163579.0     ┆ 100.0     ┆ 1.0        │
│ median     ┆ 81840.5      ┆ 98.0      ┆ 1.0        │
└────────────┴──────────────┴───────────┴────────────┘


  5%|▌         | 1/20 [05:13<1:39:21, 313.74s/it]


../data/rank_train_data_v2/train/part_2.parquet


# Eval 

In [ ]:
# ! ls {ranker_train_data_dir}

In [ ]:
# ! rm {ranker_train_data_dir}/*

In [ ]:
# pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet')).collect().shape

In [ ]:
# pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet')).collect().shape